In [16]:
import pandas as pd
import unidecode
from tqdm import tqdm
import re
import random

In [17]:
path = 'D:\Projects\VND_work\classification_fb_education\data_for_augmentation\education_train.csv'
df = pd.read_csv(path)
df

,Unnamed: 0,index,text,normed_text
0,0,13342,Trường THPTTH Phú Bình,Trường THPT Phú Bình
1,1,1143,da hoc truong trung cap quan ly nha hang khach...,trung cấp Quản trị Nhà hàng Khách sạn
2,2,12978,Trường THPT Nguyễn Du - Đồng Xoài - BP,Trường THPT Nguyễn Du
3,3,4098,ttgdnn gdtx nam dan,TT GDNN-GDTX Nam Đàn
4,4,9602,Hoc Vien An ning,Học Viện An ninh Nhân Dân
...,...,...,...,...
11691,11691,13085,Trường THPT Hồng Quang Lục Yên,Trường THPT Hồng Quang
11692,11692,7103,truong th pho thong lê hông phong,Trường THPT chuyên Lê Hồng Phong TP.HCM
11693,11693,1213,hue medical junior college,Cao Đẳng Y tế Huế
11694,11694,13872,Đại học Bách Khoa HCM,Trường Đại học Bách khoa - Đại học Quốc gia TP...


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11696 entries, 0 to 11695
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   11696 non-null  int64 
 1   index        11696 non-null  int64 
 2   text         11696 non-null  object
 3   normed_text  11696 non-null  object
dtypes: int64(2), object(2)
memory usage: 365.6+ KB


In [19]:
df['normed_text'].unique().shape

(5816,)

In [20]:
df['text'].unique().shape

(10520,)

In [21]:
df.loc[df['normed_text']=='Trường THPT Phú Bình', :]

,Unnamed: 0,index,text,normed_text
0,0,13342,Trường THPTTH Phú Bình,Trường THPT Phú Bình
8377,8377,9764,Trường THPT Phú Bình - Thái Nguyên,Trường THPT Phú Bình
10513,10513,8961,Trường THPT Phú Bình - Thái Nguyên,Trường THPT Phú Bình
11244,11244,2747,Trường THPT phu binh,Trường THPT Phú Bình


In [22]:
df.loc[df['normed_text']=='Trường THPT Phú Bình', :]['text'].to_list()

['Trường THPTTH Phú Bình',
 'Trường THPT Phú Bình - Thái Nguyên',
 'Trường THPT Phú Bình - Thái Nguyên',
 'Trường THPT phu binh']

In [58]:
df2 = pd.DataFrame(df['normed_text'].unique(), columns=['normed_text_unique'])
df2

,normed_text_unique
0,Trường THPT Phú Bình
1,trung cấp Quản trị Nhà hàng Khách sạn
2,Trường THPT Nguyễn Du
3,TT GDNN-GDTX Nam Đàn
4,Học Viện An ninh Nhân Dân
...,...
5811,TT GDNN-GDTX huyện Giồng Riềng
5812,TT GDTX H.Xuân Trường
5813,TT GDTX tỉnh Bà Rịa - Vũng Tàu
5814,Trường THPT chuyên Lê Hồng Phong TP.HCM


In [59]:
results = dict() # storage

def string_punctuation(text):
    res = re.findall(r'\w+', text)
    return ' '.join(res)

def remove_accent(text):
    res = unidecode.unidecode(text)
    return string_punctuation(res)

def generate_augmentation(s, num_iters=1000):
    global results # storage
    s_no_accents = remove_accent(s).lower()
    results[s] = [s, s_no_accents]
    
    augmentation = {'thpt': ['thpt', 'tthpt','pt', 'ptth', 'pho thong', 'trung hoc pho thong', 'truong trung hoc pho thong', 'cap 3', 'cap 3', 'pho thong trung hoc', 'truong pho thong trung hoc'],
                    'high school': ['high school', 'highschool'],
                    'truong trung hoc pho thong': ['truong trung hoc pho thong', 'thpt', 'pt', 'ptth', 'pho thong', 'trung hoc pho thong', 'truong thpt', 'truong cap 3', 'cap 3', 'pho thong trung hoc'],
                    'truong thpt': ['truong thpt', 'thpt', 'pt', 'ptth', 'pho thong', 'trung hoc pho thong', 'truong trung hoc pho thong', 'truong cap 3', 'cap 3', 'pho thong trung hoc'],
                    'trung hoc pho thong': ['trung hoc pho thong', 'thpt', 'pt', 'ptth', 'pho thong', 'truong thpt', 'truong trung hoc pho thong', 'truong cap 3', 'cap 3', 'pho thong trung hoc'],
                    'tp hcm': ['tp hcm', 'tp.hcm', 'tp. ho chi minh', 'tphcm', 'tp ho chi minh'],
                    'tp ho chi minh': ['tp ho chi minh', 'tp hcm', 'tp.hcm', 'tp. ho chi minh', 'tphcm'],
                    'tp. ho chi minh': ['tp. ho chi minh', 'tp hcm', 'tp.hcm', 'tp ho chi minh', 'tphcm'],
                    'dai hoc': ['dai hoc', 'daj hoc', 'dh', 'truong dai hoc', 'truong dh'],
                    'truong dai hoc': ['truong dai hoc', 'dai hoc', 'daj hoc', 'dh', 'truong dh'],
                    'cao dang': ['cao dang', 'cd', 'truong cao dang'],
                    'truong cao dang': ['truong cao dang', 'cd', 'cao dang'],
                    'cao dang su pham': ['cao dang su pham', 'cdsp'],
                    'truong cao dang su pham': ['truong cao dang su pham', 'cao dang su pham', 'cdsp'],
                    'dan lap': ['dan lap', 'dl'],
                    'ptdt nt': ['ptdt nt', 'ptdtnt', 'pho thong dan toc noi tru'],
                    'ptdt bt': ['ptdt bt', 'ptdtbt', 'pho thong dan toc ban tru'],
                    'gdtx': ['gdtx', 'ttgdtx', 'giao duc thuong xuyen', 'trung tam gdtx', 'trung tam giao duc thuong xuyen'],
                    'trung tam gdtx': ['trung tam gdtx', 'gdtx', 'ttgdtx', 'giao duc thuong xuyen', 'trung tam giao duc thuong xuyen'],
                    'trung tam giao duc thuong xuyen': ['trung tam giao duc thuong xuyen', 'trung tam gdtx', 'gdtx', 'ttgdtx', 'giao duc thuong xuyen'],
                    'giao duc thuong xuyen': ['giao duc thuong xuyen', 'gdtx', 'ttgdtx', 'trung tam gdtx', 'trung tam giao duc thuong xuyen'],
                    'bach khoa': ['bach khoa', 'bk'],
                    'dai hoc bach khoa ha noi': ['dai hoc bach khoa ha noi', 'dhbkhn'],
                    'hoc vien ngan hang': ['hoc vien ngan hang', 'hvnh'],
                    'giao thong van tai': ['giao thong van tai', 'gtvt'],
                    'truong dai hoc su pham': ['truong dai hoc su pham', 'dhsp', 'dai hoc su pham'],
                    'dai hoc su pham': ['dai hoc su pham', 'truong dai hoc su pham', 'dhsp']}

    for _ in range(num_iters):
        for k in augmentation.keys():
            if k in s_no_accents:
                index = s_no_accents.find(k)
                s_tmp = s_no_accents.replace(k, '', 1) # replace the first occurence
                n = random.randrange(len(augmentation[k]))
                insert_part = augmentation[k][n]
                one_case = s_tmp[:index] + insert_part + s_tmp[index:]
                if one_case not in results[s]:  
                    results[s].append(one_case)

In [60]:
for i in tqdm(range(len(df2)), desc="Augmentation"):
    text = df2['normed_text_unique'][i]
    generate_augmentation(text, num_iters=1000)

Augmentation: 100%|███████████████████████████████████████████████████████████████| 5816/5816 [00:45<00:00, 126.46it/s]


In [61]:
df2['augmented_name'] = df2['normed_text_unique'].apply(lambda x: results[x])
df2['count'] = df2['normed_text_unique'].apply(lambda x: len(results[x]))
df2

,normed_text_unique,augmented_name,count
0,Trường THPT Phú Bình,"[Trường THPT Phú Bình, truong thpt phu binh, t...",18
1,trung cấp Quản trị Nhà hàng Khách sạn,"[trung cấp Quản trị Nhà hàng Khách sạn, trung ...",2
2,Trường THPT Nguyễn Du,"[Trường THPT Nguyễn Du, truong thpt nguyen du,...",18
3,TT GDNN-GDTX Nam Đàn,"[TT GDNN-GDTX Nam Đàn, tt gdnn gdtx nam dan, t...",6
4,Học Viện An ninh Nhân Dân,"[Học Viện An ninh Nhân Dân, hoc vien an ninh n...",2
...,...,...,...
5811,TT GDNN-GDTX huyện Giồng Riềng,"[TT GDNN-GDTX huyện Giồng Riềng, tt gdnn gdtx ...",6
5812,TT GDTX H.Xuân Trường,"[TT GDTX H.Xuân Trường, tt gdtx h xuan truong,...",6
5813,TT GDTX tỉnh Bà Rịa - Vũng Tàu,"[TT GDTX tỉnh Bà Rịa - Vũng Tàu, tt gdtx tinh ...",6
5814,Trường THPT chuyên Lê Hồng Phong TP.HCM,"[Trường THPT chuyên Lê Hồng Phong TP.HCM, truo...",22


In [62]:
df2.loc[0, 'normed_text_unique']

'Trường THPT Phú Bình'

In [63]:
for i in tqdm(range(len(df2))):
    content = df2.loc[i, 'normed_text_unique']
    original_list = df.loc[df['normed_text']==content, :]['text'].to_list()
    df2.loc[i, 'augmented_name'].extend(original_list)
    df2.loc[i, 'count'] += len(original_list)

100%|█████████████████████████████████████████████████████████████████████████████| 5816/5816 [00:15<00:00, 373.83it/s]


In [65]:
df2

,normed_text_unique,augmented_name,count
0,Trường THPT Phú Bình,"[Trường THPT Phú Bình, truong thpt phu binh, t...",22
1,trung cấp Quản trị Nhà hàng Khách sạn,"[trung cấp Quản trị Nhà hàng Khách sạn, trung ...",3
2,Trường THPT Nguyễn Du,"[Trường THPT Nguyễn Du, truong thpt nguyen du,...",53
3,TT GDNN-GDTX Nam Đàn,"[TT GDNN-GDTX Nam Đàn, tt gdnn gdtx nam dan, t...",7
4,Học Viện An ninh Nhân Dân,"[Học Viện An ninh Nhân Dân, hoc vien an ninh n...",16
...,...,...,...
5811,TT GDNN-GDTX huyện Giồng Riềng,"[TT GDNN-GDTX huyện Giồng Riềng, tt gdnn gdtx ...",7
5812,TT GDTX H.Xuân Trường,"[TT GDTX H.Xuân Trường, tt gdtx h xuan truong,...",7
5813,TT GDTX tỉnh Bà Rịa - Vũng Tàu,"[TT GDTX tỉnh Bà Rịa - Vũng Tàu, tt gdtx tinh ...",7
5814,Trường THPT chuyên Lê Hồng Phong TP.HCM,"[Trường THPT chuyên Lê Hồng Phong TP.HCM, truo...",23


In [66]:
df2['count'].sum()

71059

In [67]:
n_rows = df2['count'].sum()

n_columns = 2
df3 = pd.DataFrame(index=range(n_rows), columns=range(n_columns))
i = 0
df3

,0,1
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
71054,NaN,NaN
71055,NaN,NaN
71056,NaN,NaN
71057,NaN,NaN


In [68]:
for k, v in tqdm(results.items(), desc='Flattening'):
    for element in v:
        df3[0][i] = k
        df3[1][i] = element
        i += 1

Flattening: 100%|██████████████████████████████████████████████████████████████████| 5816/5816 [01:47<00:00, 54.01it/s]


In [69]:
df3

,0,1
0,Trường THPT Phú Bình,Trường THPT Phú Bình
1,Trường THPT Phú Bình,truong thpt phu binh
2,Trường THPT Phú Bình,truong cap 3 phu binh
3,Trường THPT Phú Bình,truong trung hoc pho thong phu binh
4,Trường THPT Phú Bình,truong truong pho thong trung hoc phu binh
...,...,...
71054,TT GDNN-GDTX huyện Nậm Nhùn,tt gdnn trung tam gdtx huyen nam nhun
71055,TT GDNN-GDTX huyện Nậm Nhùn,tt gdnn trung tam giao duc thuong xuyen huyen ...
71056,TT GDNN-GDTX huyện Nậm Nhùn,tt gdnn giao duc thuong xuyen huyen nam nhun
71057,TT GDNN-GDTX huyện Nậm Nhùn,tt gdnn ttgdtx huyen nam nhun


In [70]:
df3.columns = ['normed_text', 'original_and_fake']
df3

,normed_text,original_and_fake
0,Trường THPT Phú Bình,Trường THPT Phú Bình
1,Trường THPT Phú Bình,truong thpt phu binh
2,Trường THPT Phú Bình,truong cap 3 phu binh
3,Trường THPT Phú Bình,truong trung hoc pho thong phu binh
4,Trường THPT Phú Bình,truong truong pho thong trung hoc phu binh
...,...,...
71054,TT GDNN-GDTX huyện Nậm Nhùn,tt gdnn trung tam gdtx huyen nam nhun
71055,TT GDNN-GDTX huyện Nậm Nhùn,tt gdnn trung tam giao duc thuong xuyen huyen ...
71056,TT GDNN-GDTX huyện Nậm Nhùn,tt gdnn giao duc thuong xuyen huyen nam nhun
71057,TT GDNN-GDTX huyện Nậm Nhùn,tt gdnn ttgdtx huyen nam nhun
